In [1]:
import ipywidgets as widgets;
from ipywidgets import *;
from IPython.display import display, clear_output;
from scripts.GoogleSheetsAPI import *
from scripts.readSheets import main
from scripts.Scraper import scrape_tracking
import pandas as pd

C:\Users\pgathondu\webScraping\client_secret_file.json-sheets-v4-(['https://www.googleapis.com/auth/spreadsheets'],)
['https://www.googleapis.com/auth/spreadsheets']
sheets service created successfully


In [2]:
class shipped_status:
    def __init__(self):
        self.shipment_created=[]
        self.shipped=[]
        self.order_list=[]
    def check_status(self, created, transit):
        self.shipment_created.append(created)
        self.shipped.append(transit)

    

In [3]:
diligent_status=shipped_status()

In [4]:
def on_start_button_clicked(event):
    with output:
        url='https://worldtrak.diligentusa.com/webtrakwtpd/ShipInquiry/ShipLookup.aspx?TrackType=ShipRefNo&TrackNo='
        output.clear_output
        try:
            diligent_status.order_list=getOrderList(url_widget.value, sheetTitle_widget.value)
            print(getOrderList(url_widget.value, sheetTitle_widget.value))
            print('getting order list')
        except Exception as e:
            print(e)
        url_list=list(map(lambda x: url+x, diligent_status.order_list))
        query=list(map(scrape_tracking, url_list))
        diligent_status.check_status(query[0]['shipment_created'], query[0]['shipped'])
        d={'Customer Reference': diligent_status.order_list, 'Shipment created': diligent_status.shipment_created, 'Shipped': diligent_status.shipped}
        df=pd.DataFrame(data=d)
        addData(url_widget.value, df)
        
def getOrderList(sheet_url, sheet_range):
    gsheetId=getSheetId(sheet_url)
    df=main(gsheetId,sheet_range)
    col_name=list(df.columns)
    return df[col_name[0]].tolist()

def addData(sheet_url, df):
    gsheetId=getSheetId(sheet_url)
    worksheet_name='scraped Data'
    add_sheets(gsheetId, worksheet_name)
    clearSheets(gsheetId,worksheet_name)
    writeDataToSheetDf(worksheet_name,gsheetId,df)
    

In [5]:
# widgets
project_widget = Dropdown(layout={'width': 'max-content'})
workbook_widget = Dropdown(layout={'width': 'max-content'})
worksheet_widget = Dropdown(layout={'width': 'max-content'})
t = widgets.HBox(children=[project_widget, workbook_widget,worksheet_widget])

startBtn = widgets.Button(
    description = 'Start Rest API',
    tooltip = 'Send',
    style = {'description_width': 'initial'}
)

loginBtn = widgets.Button(
    description = 'Login',
    tooltip = 'Send',
    style = {'description_width': 'initial'}
)

importBtn = widgets.Button(
    description = 'Create Export',
    tooltip = 'Send',
    style = {'description_width': 'initial'},
    layout = {'bottom': '0px'}
)


url_widget=widgets.Text(
    value='',
    placeholder='Enter Spreadsheet Link',
    description='URL Link:',
    disabled=False
)


sheetTitle_widget=widgets.Text(
    value='',
    placeholder='Enter Range Name',
    description='Range:',
    disabled=False
)

widgets.Combobox(
    # value='John',
    placeholder='Choose Someone',
    options=['Paul', 'John', 'George', 'Ringo'],
    description='Combobox:',
    ensure_option=True,
    disabled=False
)

widget_username=widgets.Text(
value='',
placeholder='Type something',
description='Username:',
disabled=False
)
widget_password=widgets.Password(
value='',
placeholder='Enter password',
description='Password:',
disabled=False
)

folder_name=widgets.HTML(value='<h3><b><p style="text-align: right;\
                        font-size: 10px;background-color:orange">\
                        BI Folder Name</p></b></h3>',
             layout=Layout(border='solid_red 1px red', width='50px', align_self= 'center')
            )
workbook_name=widgets.HTML(value='<h3><b><p style="text-align: right;\
                        font-size: 10px;background-color:orange">\
                        WorkBook Name</p></b></h3>',
             layout=Layout(border='solid_red 1px red', width='100px')
            )
view_name=widgets.HTML(value='<h3><b><p style="text-align: right;\
                        font-size: 10px;background-color:orange">\
                        BI View Name</p></b></h3>',
             layout=Layout(border='solid_red 1px red', margin='5px', width = '150px')
            )
bi_files = widgets.HBox(children=[folder_name, workbook_name, view_name])


a1 = widgets.Accordion(children=[widgets.HBox(children=[widgets.Text(), widgets.Text()])],)
a1.set_title(0, 'Dimension Name, Value')
a2 = widgets.Accordion(children=[widgets.HBox(children=[widgets.Text(), widgets.Text()])],)
a2.set_title(0, 'Dimension Name, Value')

tab_nest = widgets.Tab()
tab_nest.children = [a1, a2]
for i in range(len(tab_nest.children)):
    tab_nest.set_title(i, 'filter '+str(i+1))

scenario_widget=widgets.RadioButtons(
    options=['none','Get Diligent Template', 'Close MOs', 'Other'],
#    value='pineapple', # Defaults to 'pineapple'
    layout={'width': 'max-content', 'margin': '100px'}, # If the items' names are long
    style = {'description_width': 'initial'},
    description='Choose Scenario:',
    disabled=False
)

filter_widget=widgets.Combobox(
    # value='John',
    placeholder='Choose Ivan\'s Fiter',
    description='Ivan\'s Filter:',
    ensure_option=True,
    disabled=False
)

cacheLoadWidget = IntProgress(min=0, max=600,
            description='Loading: Clearing BI Cached View ~ 1 minute',
            bar_style='success', # 'success', 'info', 'warning', 'danger' or ''
            style={'bar_color': 'blue'},
            orientation='horizontal') # instantiate the bar

style = {'text-align: center', 'font-size: 500px'}
pre_login_header=widgets.HTML(value='<h1><b><p style="text-align: center;\
                        font-size: 20px;background-color:red">\
                        Tableau Rest API login please!</p></b></h1>',
             layout=Layout(border='solid_red 1px red', margin='5px')
            )
post_login_header=widgets.HTML(value='<h1><b><p style="text-align: center;\
                        font-size: 20px;background-color:powderblue">\
                        Tableau Rest API logged in</p></b></h1>',
             layout=Layout(border='solid_black 1px ', margin='5px')
            )

In [6]:
output = widgets.Output()
startBtn.on_click(on_start_button_clicked)
vbox_after_login = widgets.VBox(children=[post_login_header,widgets.HBox(children=[url_widget,sheetTitle_widget])])
vbox_display=widgets.VBox(children=[vbox_after_login,startBtn, output])
display(vbox_display)
